In [12]:
import numpy as np;
import time;

In [13]:
from RBComb.SerialBridge import *
from RBComb.RBComb576 import *
from RBComb.Helper.BoardPinout import *
from RBComb.Helper.BoardUnits import *;
from RBComb.Helper.FunctionInterpolator import *

In [14]:
# Construct bridge:
currentBridge = SerialBridge('/dev/tty.usbserial-FT3HPF65B', True, 300);

# Note: the correct baud rate is 5'000'000 but it is not supported in Mac OS X
# The solution is to modify the serial port driver so it aliases ("pretends") to be working at a standard
# rate of 300, but in reality it works at 5000000. This is done by modifying /Library/Extensions/FTDIUSBSerialDriver.kext
# following the guide https://www.ftdichip.com/Support/Documents/TechnicalNotes/TN_105%20Adding%20Support%20for%20New%20FTDI%20Devices%20to%20Mac%20Driver.pdf
# And subsequently disabling the "trusted platform" features of mac since it would no longer accept the modified extension

SerialException: [Errno 2] could not open port /dev/tty.usbserial-FT3HPF65B: [Errno 2] No such file or directory: '/dev/tty.usbserial-FT3HPF65B'

In [15]:
# Construct RBComb576
rbComb = RBComb576(currentBridge);

In [16]:
# Construct helper objects:
pinout = BoardPinout();
units = BoardUnits();

In [ ]:
# We must make sure the clock is disabled, otherwise the boards cannot hear us
rbComb.disableClock()

# Select all boards
rbComb.selectAllBoards()

In [ ]:
# We use the interpolator to raise the signal to V/2 as the output cannot take negative values
interpolator = FunctionInterpolator();
constantFunction = interpolator.getConstantFunction(0.5)

# Upload offset function to the device (Takes 10 seconds, do only once per "session")
for i in range(576):
    rbComb.setTaylorCoefficients(i, False, constantFunction);

In [ ]:
# Parameters for the sweep
frequencySweepStart = 500;   # 1 KHz
frequencySweepEnd =  5000;    # 5 KHz
sweepDuration = 40e-3;        # 20 msec
amplitudes = np.array([units.convertNormalizedAmplitudeToInteger(1.0)]*576); # We set all channels to max amplitude (could also use different amps per channel).

In [ ]:
# Configure amplitudes
for i in range(576):
    rbComb.setAmplitude(0,i,amplitudes[i]);
    rbComb.setAmplitude(1,i,0);
    rbComb.setAmplitude(2,i,0);
    rbComb.setAmplitude(3,i,0);

In [ ]:
# Configure frequencies and phases:

# Start frequency
rbComb.setFrequency(0,units.convertFrequencyHzToInteger(frequencySweepStart));


# Frequency step>:
rbComb.setSweepFrequencyStep(0,units.convertFrequencyRangeToSweepSpeed(frequencySweepStart, 
                                                                       frequencySweepEnd, sweepDuration));

# Frequency end:
rbComb.setSweepFrequencyMax(0,units.convertFrequencyHzToInteger(frequencySweepEnd));

rbComb.setPhase(0,0);
rbComb.setPhase(1,0);
rbComb.setPhase(2,0);
rbComb.setPhase(3,0);

In [ ]:
# Ramp up the output up with the generators "frozen":
rbComb.outputRampUp();
rbComb.setCurrentState([0,0,0,0,0]);

rbComb.enableClock()
time.sleep(4);
rbComb.disableClock()

In [ ]:
# Run experiment

# Unfreeze first generator:
rbComb.setCurrentState([1,0,0,0,0]);

# Schedule re-freezing of the first generator once the sweep is finished
rbComb.setNextState([0,0,0,0,0], units.convertTimeToSteps(sweepDuration));

# Run one sweep:
rbComb.enableClock()
time.sleep(0.5);
rbComb.disableClock()

In [ ]:
# Now run 5 sweeps. This shows that once the system is set up, it is possible to run sweeps very fast
# Transients are avoided by not re-setting the phase to zero after each sweep.
for i in range(5):
    # Unfreeze first generator:
    rbComb.setCurrentState([1,0,0,0,0]);

    # Schedule re-freezing of the first generator once the sweep is finished
    rbComb.setNextState([0,0,0,0,0], units.convertTimeToSteps(sweepDuration));

    # Go back to the initial frequency:
    rbComb.setFrequency(0,units.convertFrequencyHzToInteger(frequencySweepStart));
    
    # Run the sweep:
    rbComb.enableClock()
    time.sleep(0.1);
    rbComb.disableClock()

In [ ]:
# Ramp down the output
rbComb.outputRampDown();
rbComb.enableClock()
time.sleep(4);
rbComb.disableClock()

In [11]:
currentBridge.closeConnection()

In [9]:
rbComb.enableClock();
rbComb.selectAllBoards();